In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

df_data = [
    ('A', 'Good', 9.5, 'horror'),
    ('B', 'Good', 8.2, 'Comedy'),
    ('C', 'Avg', 9.5, 'Comedy'),
    ('D', 'Avg', 9.5, 'horror')
]
df_columns = ["title", "reviews", "ratings", "categories"]
df = spark.createDataFrame(df_data, df_columns)

casting_df_data = [
    ('Actor1', 'A', 9.0),
    ('Actor2', 'B', 8.5),
    ('Actor3', 'C', 7.5),
    ('Actor4', 'D', 8.0)
]
casting_df_columns = ["actor_name", "movie_name", "actor_rating"]

casting_df = spark.createDataFrame(casting_df_data, casting_df_columns)



questions
Q1) join df and casting_df on movie name.
Q2) Find the category of the actor and movie.
Q3) Find the what is the avg rating of actor
Q4) how to sort in pyspark?


In [0]:
#q1

joined_df=df.join(casting_df,df.title==casting_df.movie_name,"inner")
joined_df.show()

+-----+-------+-------+----------+----------+----------+------------+
|title|reviews|ratings|categories|actor_name|movie_name|actor_rating|
+-----+-------+-------+----------+----------+----------+------------+
|    A|   Good|    9.5|    horror|    Actor1|         A|         9.0|
|    B|   Good|    8.2|    Comedy|    Actor2|         B|         8.5|
|    C|    Avg|    9.5|    Comedy|    Actor3|         C|         7.5|
|    D|    Avg|    9.5|    horror|    Actor4|         D|         8.0|
+-----+-------+-------+----------+----------+----------+------------+



In [0]:
#q2

joined_df.select("title","categories","actor_name").show()

+-----+----------+----------+
|title|categories|actor_name|
+-----+----------+----------+
|    A|    horror|    Actor1|
|    B|    Comedy|    Actor2|
|    C|    Comedy|    Actor3|
|    D|    horror|    Actor4|
+-----+----------+----------+



In [0]:
#q3
joined_df.agg(avg("actor_rating")).show()

+-----------------+
|avg(actor_rating)|
+-----------------+
|             8.25|
+-----------------+



In [0]:
#q4
joined_df.orderBy("ratings", ascending=False).show()

+-----+-------+-------+----------+----------+----------+------------+
|title|reviews|ratings|categories|actor_name|movie_name|actor_rating|
+-----+-------+-------+----------+----------+----------+------------+
|    A|   Good|    9.5|    horror|    Actor1|         A|         9.0|
|    C|    Avg|    9.5|    Comedy|    Actor3|         C|         7.5|
|    D|    Avg|    9.5|    horror|    Actor4|         D|         8.0|
|    B|   Good|    8.2|    Comedy|    Actor2|         B|         8.5|
+-----+-------+-------+----------+----------+----------+------------+



In [0]:
data = [
    (1, "Alice", None),
    (2, "Bob", 1),
    (3, "Carol", 2),
    (4, "Dave", 1),
    (5, "Eve", 2),
    (6, "Frank", 4)
]

schema = ["ID", "Name", "Boss"]


df = spark.createDataFrame(data, schema)
df.show()

+---+-----+----+
| ID| Name|Boss|
+---+-----+----+
|  1|Alice|NULL|
|  2|  Bob|   1|
|  3|Carol|   2|
|  4| Dave|   1|
|  5|  Eve|   2|
|  6|Frank|   4|
+---+-----+----+



In [0]:
#Self Join
df_joined=df.alias("df1").join(df.alias("df2"), col("df1.Boss") == col("df2.ID"),"left")\
    .select(col("df1.ID"),col("df1.Name"), col("df2.Name").alias("Boss_name"))

#Fill null values with "No Boss"
df_joined=df_joined.fillna("No Boss",subset=["Boss_name"])
df_joined.show()

+---+-----+---------+
| ID| Name|Boss_name|
+---+-----+---------+
|  1|Alice|  No Boss|
|  2|  Bob|    Alice|
|  3|Carol|      Bob|
|  4| Dave|    Alice|
|  5|  Eve|      Bob|
|  6|Frank|     Dave|
+---+-----+---------+



In [0]:
data=[(1,"Gaurav","Pune,hyd,Blr"),(2,"Ravi","hyd,Blr")]

columns=["Empid","Name","Locations"]
df_emp=spark.createDataFrame(data,columns)

df_emp.show()

+-----+------+------------+
|Empid|  Name|   Locations|
+-----+------+------------+
|    1|Gaurav|Pune,hyd,Blr|
|    2|  Ravi|     hyd,Blr|
+-----+------+------------+



Non Bengalore 

In [0]:

df1=df_emp.select("Empid","Name", explode(split(col("Locations"),",")))\
  .withColumnRenamed("col","Location").filter(col("Location")!="Blr")
df1.show()

+-----+------+--------+
|Empid|  Name|Location|
+-----+------+--------+
|    1|Gaurav|    Pune|
|    1|Gaurav|     hyd|
|    2|  Ravi|     hyd|
+-----+------+--------+



count the number of movies in each genre

In [0]:

df = spark.createDataFrame([('The Shawshank Redemption',['Drama', 'Crime']),
                  ('The Godfather', ['Drama', 'Crime']),
                  ('Pulp Fiction', ['Drama', 'Crime','Thriller']),
                  ('The Dark Knight', ['Drama', 'Crime','Thriller','Action']),
                  ],["name", "genres"])

df1=df.select("name",explode(col("genres")).alias("genres"))

df1.groupBy("genres").agg(count("name")).show()

+--------+-----------+
|  genres|count(name)|
+--------+-----------+
|   Crime|          4|
|   Drama|          4|
|Thriller|          2|
|  Action|          1|
+--------+-----------+



Question: Calculate the month-wise cumulative revenue using PySpark?


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark=SparkSession.builder.getOrCreate()

spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY" )
data = [( 
'3000' ,  '22-may'), 
('5000' ,  '23-may'),
('5000' ,  '25-may'),
('10000' , '22-june'),  
('1250'  , '03-july')]

schema = ['revenue','date']
df_rev=spark.createDataFrame(data,schema)


spark=SparkSession.builder.getOrCreate()

spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY" )

In [0]:
df=df_rev.withColumn("month",date_format(to_date(col("date"),"dd-MMM"),"MMM"))

window_spec=Window.partitionBy('month').orderBy('date').rowsBetween(Window.unboundedPreceding,0)

df=df.withColumn('cumulative_sum',sum(col("revenue")).over(window_spec))

df.show()

+-------+-------+-----+--------------+
|revenue|   date|month|cumulative_sum|
+-------+-------+-----+--------------+
|   1250|03-JULY|  Jul|        1250.0|
|  10000|22-JUNE|  Jun|       10000.0|
|   3000| 22-MAY|  May|        3000.0|
|   5000| 23-MAY|  May|        8000.0|
|   5000| 25-MAY|  May|       13000.0|
+-------+-------+-----+--------------+



In [0]:
from pyspark.sql.functions import col, date_format, to_date, sum, window, initcap

windowSpec = Window.partitionBy("month").orderBy("month").rowsBetween(Window.unboundedPreceding, Window.currentRow)
df_rev = df_rev.withColumn("date", upper(col("date")))  # Capitalize the first letter
df_rev = df_rev.withColumn("month", date_format(to_date(col("date"), "dd-MMM"), "MMM"))
df_final = df_rev.withColumn("run_revenue", sum("revenue").over(windowSpec))
display(df_final)

revenue,date,month,run_revenue
1250,03-JULY,Jul,1250.0
10000,22-JUNE,Jun,10000.0
3000,22-MAY,May,3000.0
5000,23-MAY,May,8000.0
5000,25-MAY,May,13000.0


In [0]:
data = [('4529', 'Nancy', 'Young', '4125'),
('4238','John', 'Simon', '4329'),
('4329', 'Martina', 'Candreva', '4125'),
('4009', 'Klaus', 'Koch', '4329'),
('4125', 'Mafalda', 'Ranieri', 'NULL'),
('4500', 'Jakub', 'Hrabal', '4529'),
('4118', 'Moira', 'Areas', '4952'),
('4012', 'Jon', 'NULL', '4952'),
('4952', 'Sandra', 'Rajkovic', '4529'),
('4444', 'Seamus', 'Quinn', '4329')]

schema = ['employee_id' ,'first_name', 'last_name', 'manager_id']

df=spark.createDataFrame(data, schema)
df.show()

+-----------+----------+---------+----------+
|employee_id|first_name|last_name|manager_id|
+-----------+----------+---------+----------+
|       4529|     Nancy|    Young|      4125|
|       4238|      John|    Simon|      4329|
|       4329|   Martina| Candreva|      4125|
|       4009|     Klaus|     Koch|      4329|
|       4125|   Mafalda|  Ranieri|      NULL|
|       4500|     Jakub|   Hrabal|      4529|
|       4118|     Moira|    Areas|      4952|
|       4012|       Jon|  Nilssen|      4952|
|       4952|    Sandra| Rajkovic|      4529|
|       4444|    Seamus|    Quinn|      4329|
+-----------+----------+---------+----------+



In [0]:
df_new=df.alias("e").join(df.alias("m"), col("e.manager_id") == col("m.employee_id"),"inner").select(col("e.employee_id"),col("e.first_name"), col("m.first_name").alias("manager"))

df_new.groupBy("manager").count().alias("no_of_emp").show()


+-------+-----+
|manager|count|
+-------+-----+
|  Nancy|    2|
|Martina|    3|
|Mafalda|    2|
| Sandra|    2|
+-------+-----+



In [0]:
df.createOrReplaceTempView("emp")
query = '''select  e.manager_id as manager_id, 
count(e.employee_id) as no_of_emp,(m.First_name) as mangr_name 
from  emp e
inner join emp m 
on m.employee_id =e.manager_id
group by 1,3
'''
result=spark.sql(query).show()

+----------+---------+----------+
|manager_id|no_of_emp|mangr_name|
+----------+---------+----------+
|      4529|        2|     Nancy|
|      4329|        3|   Martina|
|      4125|        2|   Mafalda|
|      4952|        2|    Sandra|
+----------+---------+----------+



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
df=df.withColumn("employee_id",col("employee_id").cast("int"))\
    .withColumn("manager_id",col("manager_id").cast("int"))
df.printSchema()    

root
 |-- employee_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- manager_id: integer (nullable = true)



In [0]:
from pyspark.sql.functions import col

df1 = df.select([count(col(i)) for i in df.columns])

df1.show()

---------------------------------------------------------------------------
NumberFormatException                     Traceback (most recent call last)
File <command-2436434663360693>, line 5
      1 from pyspark.sql.functions import col
      3 df1 = df.select([count(col(i)) for i in df.columns])
----> 5 df1.show()

File /databricks/python/lib/python3.10/site-packages/pyspark/sql/connect/dataframe.py:1159, in DataFrame.show(self, n, truncate, vertical)
   1158 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
-> 1159     print(self._show_string(n, truncate, vertical))

File /databricks/python/lib/python3.10/site-packages/pyspark/sql/connect/dataframe.py:903, in DataFrame._show_string(self, n, truncate, vertical)
    886     except ValueError:
    887         raise PySparkTypeError(
    888             error_class="NOT_BOOL",
    889             message_parameters={
   (...)
    892             },
    893         )
    895 table, _ = DataFr

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import *
data = [1, 2, 3, 4, 6, 8, 10]
schema = StructType([StructField("id", IntegerType(), True)])

df=spark.createDataFrame(data, schema)

In [0]:
#Finding missing ID.
minval=df.select(min(col('id'))).collect()[0][0]
maxval=df.select(max(col('id'))).collect()[0][0]
df1=spark.range(minval,maxdf+1)
df1.show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
+---+



In [0]:
df_missing=df1.join(df, df1.id==df.id, how='left_anti')
df_missing.show()

+---+
| id|
+---+
|  5|
|  7|
|  9|
+---+

